In [ ]:
!pip install selenium
!pip install selenium_stealth
!pip install selenium webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 9.4 MB/s eta 0:00:00


In [ ]:
# import libraries
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import pandas as pd
import numpy as np
import re
import time
import random
import os

In [ ]:
# Initialize Selenium WebDriver with options
def initialize_driver_options():
    # create a ChromeOptions object
    options = Options()

    #run in headless mode
    options.add_argument("--headless")

    # disable the AutomationControlled feature of Blink rendering engine
    options.add_argument('--disable-blink-features=AutomationControlled')

    # disable pop-up blocking
    options.add_argument('--disable-popup-blocking')

    # start the browser window in maximized mode
    options.add_argument('--start-maximized')

    # disable extensions
    options.add_argument('--disable-extensions')

    # disable sandbox mode
    options.add_argument('--no-sandbox')

    # disable shared memory usage
    options.add_argument('--disable-dev-shm-usage')


    return options

In [ ]:
# user agents list
user_agent_list = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
                   'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
                  ]
# initialise driver options
options = initialize_driver_options()

# request for a random and different user agent everytime
user_agent = random.choice(user_agent_list)

options.add_argument(f'user-agent={user_agent}')

#initialise the fortified scraper
driver = webdriver.Chrome(options=options)

driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": user_agent})

#enable stealth mode
stealth(driver,
    languages=["en-US", "en"],
    vendor="Google Inc.",
    platform="Win32",
    webgl_vendor="Intel Inc.",
    renderer="Intel Iris OpenGL Engine",
    fix_hairline=True,
      )


In [ ]:
# get desired category links
def get_cat_links(site):
  driver.get(site)
  time.sleep(2)
  links_list = []
  #fetch the navigation links in the bbc igbo site
  for el in driver.find_elements(By.XPATH, '/html/body/div[1]/div/header/nav/div/div[1]/div/ul'):
    for link in el.find_elements(By.TAG_NAME, 'a'):
      links = link.get_attribute('href')
      links_list.append(links)

  return links_list[1:3]

links = get_cat_links("https://www.bbc.com/igbo")
links

['https://www.bbc.com/igbo/topics/cnq68k0x2vrt',
 'https://www.bbc.com/igbo/topics/c3l19z3qjmyt']

In [ ]:
# function to get no. of pages
def get_np(link, id):
  driver.get(links[id])
  time.sleep(2)
  page = ""
  for pages in driver.find_elements(By.XPATH, '/html/body/div[1]/div/div/main/div/nav/ul'):
    p = pages.text
    page += p
  pages = page.split()[-1]
  return pages

In [ ]:
# function to scrape each category
def scrape(id):
  # id 0 signifies Egwuregwu category
  # id 1 signifies Ihe nkiri category
  # dictionary to store scraped data
  dic = {"text":[], "category": [], "source": []}

  pages = get_np(links, id)
  #for page in range(1,int(pages)+1):
  for page in range(10,16): #scrape five pages simultaneously at different times to reduce traffic
    # scrape the desired categories
    driver.get(links[id]+"?page="+str(page))
    time.sleep(2)
    link_list = []

    # get category name
    category = driver.find_element(By.TAG_NAME, "h1").text

    # get category articles links
    for el in driver.find_elements(By.XPATH, '/html/body/div[1]/div/div/main/div/div[2]/ul'):
        for link in el.find_elements(By.TAG_NAME, 'a'):
          pages = link.get_attribute('href')
          link_list.append(pages)

    #extract data from articles in the page
    for i in link_list:
      driver.get(i)
      time.sleep(2)
      article = ""
      for el in driver.find_elements(By.ID, "main-wrapper"):
        for p in el.find_elements(By.TAG_NAME, "main"):
          article += p.text
          article += "\n"
      dic["text"].append(article)
      dic["category"].append(category)
      dic["source"].append(i)
    print("Extraction of news content from "+category+" category page " + str(page) + " successfull!")
  driver.quit()
  return dic


In [ ]:
# scrape Egwuregwu category (0)
#dic = scrape(0)

Extraction of news content from Egwuruegwu category page 20 successfull!
Extraction of news content from Egwuruegwu category page 21 successfull!
Extraction of news content from Egwuruegwu category page 22 successfull!
Extraction of news content from Egwuruegwu category page 23 successfull!


In [ ]:
# scrape Ihe Nkiri category (1)
dic = scrape(1)

Extraction of news content from Ihe nkiri category page 10 successfull!
Extraction of news content from Ihe nkiri category page 11 successfull!
Extraction of news content from Ihe nkiri category page 12 successfull!
Extraction of news content from Ihe nkiri category page 13 successfull!
Extraction of news content from Ihe nkiri category page 14 successfull!
Extraction of news content from Ihe nkiri category page 15 successfull!


In [ ]:
df = pd.DataFrame(dic)
df

,text,category,source
0,"Peter Obi: Ihe ndị ị maghị maka Obi, Tinubu, P...",Ihe nkiri,https://www.bbc.com/igbo/articles/ce93gmgv75po
1,New Naira Scarcity: Ihe ndị mmadụ na-agabiga n...,Ihe nkiri,https://www.bbc.com/igbo/articles/c3gn7gzel8po
2,Sex for work: Etu ndịisi ụlọọrụ ebe a na-akọpụ...,Ihe nkiri,https://www.bbc.com/igbo/articles/cgrngd42r9zo
3,First-time voters: 'Ihe mere ntuliaka 2023 a j...,Ihe nkiri,https://www.bbc.com/igbo/articles/c4nzzmyzzvko
4,Nigeria Election 2023: Ihe i nwereike ime ka ị...,Ihe nkiri,https://www.bbc.com/igbo/articles/c72lqpx1e00o
...,...,...,...
121,Chimmie Gbugu: Nye ụmụ gị ihe niile ha ga-eji ...,Ihe nkiri,https://www.bbc.com/igbo/afirika-46445319
122,Anaghị ere nwaanyị ere\nLIVE\nSTART\nLIVE\nVid...,Ihe nkiri,https://www.bbc.com/igbo/afirika-45640327
123,Biafra: Okey Ndibe kọrọ ihe ọ hụrụ dịka nwata ...,Ihe nkiri,https://www.bbc.com/igbo/afirika-44736373
124,Kedụ aha otutu ị na-etu onye ị hụrụ n'anya?\nL...,Ihe nkiri,https://www.bbc.com/igbo/afirika-43662223


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      126 non-null    object
 1   category  126 non-null    object
 2   source    126 non-null    object
dtypes: object(3)
memory usage: 3.1+ KB


In [ ]:
#df.to_csv("igbo_news_data1.csv", encoding='utf-8')

In [ ]:
data = pd.read_csv("igbo_news_data.csv", index_col=False)
new_dic = pd.concat([data, df])

In [ ]:
new_dic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 765 entries, 0 to 125
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      765 non-null    object
 1   category  765 non-null    object
 2   source    765 non-null    object
dtypes: object(3)
memory usage: 23.9+ KB


In [ ]:
new_dic.to_csv("igbo_news_data.csv", index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Data

/content/drive/MyDrive/Data


In [ ]:
!zip -r scraped_data.zip new_dic/

In [ ]:
from google.colab import files
files.download('scraped_data.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>